In [33]:
import numpy as np 
import pandas as pd 
import re
import string
import nltk
from nltk.corpus import stopwords
from collections import Counter
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow import keras
from tensorflow.keras import layers

# 1.Data View

In [34]:
train = pd.read_csv("/content/drive/MyDrive/Data/train.csv")
test = pd.read_csv('/content/drive/MyDrive/Data/test.csv')

In [35]:
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [36]:
train.shape

(7613, 5)

In [37]:
print((train.target == 1).sum()) # Disaster
print((train.target == 0).sum()) # No Disaster

3271
4342


# 2.Preprocessing

In [38]:
def remove_URL(text):
    url = re.compile(r"https?://\S+|www\.\S+")
    return url.sub(r"", text)

def remove_punct(text):
    translator = str.maketrans("", "", string.punctuation)
    return text.translate(translator)

In [39]:
#regex pattern to remove links
pattern = re.compile(r"https?://(\S+|www)\.\S+")
#for train
for t in train.text:
    matches = pattern.findall(t)
    for match in matches:
        print(t)
        print('After Transformation:')
        print(pattern.sub(r"", t))
    if len(matches) > 0:
        break

@bbcmtd Wholesale Markets ablaze http://t.co/lHYXEOHY6C
After Transformation:
@bbcmtd Wholesale Markets ablaze 


In [40]:
#for test:
for t in test.text:
    matches = pattern.findall(t)
    for match in matches:
        print(t)
        print('After Transformation:')
        print(pattern.sub(r"", t))
    if len(matches) > 0:
        break

Birmingham Wholesale Market is ablaze BBC News - Fire breaks out at Birmingham's Wholesale Market http://t.co/irWqCEZWEU
After Transformation:
Birmingham Wholesale Market is ablaze BBC News - Fire breaks out at Birmingham's Wholesale Market 


In [41]:
#preprocess data frames:
#train
train["text"] = train.text.map(remove_URL) 
train["text"] = train.text.map(remove_punct)
#test
test["text"] = test.text.map(remove_URL) 
test["text"] = test.text.map(remove_punct)

In [42]:
# remove stopwords
nltk.download('stopwords')

stop = set(stopwords.words("english"))

def remove_stopwords(text):
    filtered_words = [word.lower() for word in text.split() if word.lower() not in stop]
    return " ".join(filtered_words)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [43]:
#train
train["text"] = train.text.map(remove_stopwords)
#test
test["text"] = test.text.map(remove_stopwords)

In [44]:
#Check
train.text

0            deeds reason earthquake may allah forgive us
1                   forest fire near la ronge sask canada
2       residents asked shelter place notified officer...
3       13000 people receive wildfires evacuation orde...
4       got sent photo ruby alaska smoke wildfires pou...
                              ...                        
7608    two giant cranes holding bridge collapse nearb...
7609    ariaahrary thetawniest control wild fires cali...
7610                      m194 0104 utc5km volcano hawaii
7611    police investigating ebike collided car little...
7612    latest homes razed northern california wildfir...
Name: text, Length: 7613, dtype: object

In [45]:
# Count unique words
def counter_word(text_col):
    count = Counter()
    for text in text_col.values:
        for word in text.split():
            count[word] += 1
    return count


counter = counter_word(train.text)

In [46]:
len(counter)

17971

In [47]:
counter.most_common(5)

[('like', 345), ('im', 299), ('amp', 298), ('fire', 250), ('get', 229)]

In [48]:
num_unique_words = len(counter)
num_unique_words

17971

In [49]:
# Split dataset into training and validation set
X = train.text
y = train.target
train_sentences, val_sentences , train_labels, val_labels = train_test_split(X, y, test_size=0.2)

In [50]:
#train/val
train_sentences = train_sentences.to_numpy()
train_labels = train_labels.to_numpy()
val_sentences = val_sentences.to_numpy()
val_labels = val_labels.to_numpy()

In [51]:
#test
test_sentences = test.text.to_numpy()

In [52]:
train_sentences.shape, val_sentences.shape

((6090,), (1523,))

In [53]:
# Tokenize
# vectorize a text corpus by turning each text into a sequence of integers

tokenizer = Tokenizer(num_words=num_unique_words)
tokenizer.fit_on_texts(train_sentences) # fit only to training

In [54]:
# Now each word has unique index
word_index = tokenizer.word_index
word_index

{'like': 1,
 'amp': 2,
 'im': 3,
 'fire': 4,
 'get': 5,
 'new': 6,
 'via': 7,
 'news': 8,
 'people': 9,
 'dont': 10,
 'one': 11,
 'video': 12,
 '2': 13,
 'us': 14,
 'emergency': 15,
 'disaster': 16,
 'police': 17,
 'still': 18,
 'would': 19,
 'suicide': 20,
 'body': 21,
 'crash': 22,
 'california': 23,
 'burning': 24,
 'storm': 25,
 'back': 26,
 'know': 27,
 'world': 28,
 'day': 29,
 'bomb': 30,
 'buildings': 31,
 'man': 32,
 'rt': 33,
 'see': 34,
 'time': 35,
 'got': 36,
 'go': 37,
 'attack': 38,
 'fires': 39,
 'going': 40,
 'two': 41,
 'love': 42,
 'cant': 43,
 'war': 44,
 'nuclear': 45,
 'youtube': 46,
 'first': 47,
 '3': 48,
 'full': 49,
 'car': 50,
 'may': 51,
 'families': 52,
 'train': 53,
 'killed': 54,
 'good': 55,
 'last': 56,
 'hiroshima': 57,
 'many': 58,
 'think': 59,
 'u': 60,
 'want': 61,
 'way': 62,
 'dead': 63,
 'today': 64,
 'years': 65,
 'say': 66,
 'could': 67,
 'accident': 68,
 'home': 69,
 'life': 70,
 'mass': 71,
 'even': 72,
 'collapse': 73,
 'wildfire': 74,
 'ne

In [55]:
#apply on train, validation, and test sentences

train_sequences = tokenizer.texts_to_sequences(train_sentences)
val_sequences = tokenizer.texts_to_sequences(val_sentences)
test_sequences = tokenizer.texts_to_sequences(test_sentences)

In [56]:
#Check
print(train_sentences[10:15])
print(train_sequences[10:15])

['pretty teen hayden ryan poses strips purple top view download video'
 'perkpearl thats id traumatised ok car gone gbbo relax'
 'zhenghxn tried 11 eyes akame ga kill tokyo ghoul damn bloody dont dare watch'
 'world war ii book lightning joe autobiography general j lawton collins'
 'hot funtenna hijacking computers send data sound waves black hat 2015 prebreak best']
[[878, 1201, 5747, 2823, 5748, 5749, 1913, 111, 726, 879, 12], [5750, 168, 398, 427, 319, 50, 880, 727, 2255], [5751, 824, 677, 606, 5752, 2824, 484, 3718, 5753, 569, 188, 10, 2256, 76], [28, 44, 1666, 521, 235, 1449, 3719, 935, 2257, 3720, 3721], [131, 995, 570, 825, 428, 678, 459, 728, 116, 776, 120, 607, 89]]


In [57]:
# Pad the sequences to have the same length
max_length = 15 #arbitrary number

train_padded = pad_sequences(train_sequences, maxlen=max_length, padding="post", truncating="post") #post-> 0
val_padded = pad_sequences(val_sequences, maxlen=max_length, padding="post", truncating="post")
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding="post", truncating="post")

In [58]:
#Check
train_padded.shape, val_padded.shape

((6090, 15), (1523, 15))

In [59]:
#Check
train_padded[10]

array([ 878, 1201, 5747, 2823, 5748, 5749, 1913,  111,  726,  879,   12,
          0,    0,    0,    0], dtype=int32)

In [60]:
#Check
print(train_sentences[10])
print(train_sequences[10])
print(train_padded[10])

pretty teen hayden ryan poses strips purple top view download video
[878, 1201, 5747, 2823, 5748, 5749, 1913, 111, 726, 879, 12]
[ 878 1201 5747 2823 5748 5749 1913  111  726  879   12    0    0    0
    0]


In [61]:
# flip (key, value)
reverse_word_index = dict([(idx, word) for (word, idx) in word_index.items()])

In [62]:
#decoding
def decode(sequence):
    return " ".join([reverse_word_index.get(idx, "?") for idx in sequence])

In [63]:
decoded_text = decode(train_sequences[10])
#Check
print(train_sequences[10])
print(decoded_text)

[878, 1201, 5747, 2823, 5748, 5749, 1913, 111, 726, 879, 12]
pretty teen hayden ryan poses strips purple top view download video


# 3.Training model

In [64]:
# Create LSTM model

# Embedding: Turns positive integers (indexes) into dense vectors of fixed size.

model = keras.models.Sequential()
model.add(layers.Embedding(num_unique_words, 100, input_length=max_length))

model.add(layers.LSTM(32, dropout=0.25))
model.add(layers.Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 15, 100)           1797100   
                                                                 
 lstm (LSTM)                 (None, 32)                17024     
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 1,814,157
Trainable params: 1,814,157
Non-trainable params: 0
_________________________________________________________________


In [65]:
loss = keras.losses.BinaryCrossentropy(from_logits=False)
optim = keras.optimizers.Adam(learning_rate=0.001)
metrics = ["accuracy"]

model.compile(loss=loss, optimizer=optim, metrics=metrics)

In [66]:
model.fit(train_padded, train_labels, epochs=40, validation_data=(val_padded, val_labels), verbose=2)

Epoch 1/40
191/191 - 8s - loss: 0.5401 - accuracy: 0.7213 - val_loss: 0.4617 - val_accuracy: 0.8011 - 8s/epoch - 44ms/step
Epoch 2/40
191/191 - 5s - loss: 0.2794 - accuracy: 0.8959 - val_loss: 0.5381 - val_accuracy: 0.7715 - 5s/epoch - 28ms/step
Epoch 3/40
191/191 - 6s - loss: 0.1474 - accuracy: 0.9507 - val_loss: 0.6659 - val_accuracy: 0.7479 - 6s/epoch - 29ms/step
Epoch 4/40
191/191 - 5s - loss: 0.0958 - accuracy: 0.9708 - val_loss: 0.7585 - val_accuracy: 0.7623 - 5s/epoch - 28ms/step
Epoch 5/40
191/191 - 6s - loss: 0.0667 - accuracy: 0.9764 - val_loss: 0.9239 - val_accuracy: 0.7577 - 6s/epoch - 29ms/step
Epoch 6/40
191/191 - 5s - loss: 0.0473 - accuracy: 0.9816 - val_loss: 0.9942 - val_accuracy: 0.7544 - 5s/epoch - 29ms/step
Epoch 7/40
191/191 - 5s - loss: 0.0405 - accuracy: 0.9808 - val_loss: 1.1162 - val_accuracy: 0.7485 - 5s/epoch - 29ms/step
Epoch 8/40
191/191 - 6s - loss: 0.0349 - accuracy: 0.9844 - val_loss: 1.1654 - val_accuracy: 0.7544 - 6s/epoch - 29ms/step
Epoch 9/40
191/1

# 4.Testing model

In [69]:
predictions = model.predict(test_padded)
predictions = [1 if p > 0.5 else 0 for p in predictions]
predictions

[0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,


In [71]:
submission = pd.DataFrame({'id':test['id'].values.tolist(),'target':predictions})
submission.to_csv('submission.csv',index=False)